<h1 align = "center"> ARTIFICIAL INTELLIGENCE COURSE PROJECT </h1>
<h2 align = "center"> Timetable Scheduling Problem </h2>


## Muhammad Nouman Amjad

### Including libaries, declaring necessary variables & basic functions

In [1]:
import random
import numpy as np
import math

COURSE_NAMES = ["PF","IICT","DB","OS","ML","AI","WEB","Algo","PROB","CALCULAS","TBW","PAKSTD"]
LAB_NAMES = ["PF","IICT","DB","OS","AI"]
SECTIONS = ['A','B','C','D','E']
PROFESSORS_NAMES = ["Dr. Kashif Munir", "Ms. Maira Hashmi","Mr. Saad Salman","Dr. M Ali","Mr. Ahmed Hassan","Mr. Usman","Mr. Bilal Dar","Dr. Amir Hussain","Dr. Imran Ashraf","Dr. Ahmed Raza","Mr. Hassan Ali","Dr. Fahad Ahmed"]
ROOMS_NAMES = ["C-301","C-302","C-303","C-304","C-401","C-402","C-403","C-404"]
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
TIMESLOTS = ["8:30-9:50", "10:00-11:20", "11:30-12:50", "1:00-2:20", "2:30-3:50","4:00-5:15"]
ROOM_SIZE = ["Classroom","Hall"]
NUM_SEMESTERS = 3
NUM_DAYS = len(DAYS)
NUM_TIMESLOTS = len(TIMESLOTS)
NUM_ROOMS = len(ROOMS_NAMES)
NUM_PROFESSORS = len(PROFESSORS_NAMES)
NUM_SECTIONS = len(SECTIONS)
NUM_COURSES = len(COURSE_NAMES)
NUM_LABS = len(LAB_NAMES)
NUM_ROOM_SIZE = len(ROOM_SIZE)
NUM_COUSRES_PER_SEMESTER = NUM_COURSES/NUM_SEMESTERS

decodeIndex = [NUM_COURSES,1 ,NUM_SECTIONS, 120 ,NUM_PROFESSORS, NUM_DAYS, NUM_TIMESLOTS, NUM_ROOMS, 1,NUM_DAYS, NUM_TIMESLOTS, NUM_ROOMS, 1]
decodeIndex = [math.ceil(math.sqrt(number)) for number in decodeIndex]
# [4, 1, 3, 11, 4, 3, 3, 3, 1, 3, 3, 3, 1] 

csum = 0
newlist = []
for num in decodeIndex:
    csum += num
    newlist.append(csum)
    
cumuDecodeIndex = newlist
#[4, 5, 8, 19, 23, 26, 29, 32, 33, 36, 39, 42, 43]

def genNum(u,l):
    return random.randint(u, l)

def encode(number,length):
    num = bin(number)[2:] # [2:] removes the '0b' prefix
    num = num.zfill(length)
    return num

def decode(number):
    return int(str(number), 2)

In [2]:
def encodeChromosome(ind):
    chromosome = ""
    for i in range(len(ind)):
        encoded_part = encode(ind[i],decodeIndex[i])
        chromosome += encoded_part
    return chromosome

def decodeChromosome(ind):
    chromosome = []
    startIndex = 0
    endIndex = 0
    for i in range(len(cumuDecodeIndex)):
        if i == 0:
            startIndex = 0
        else:
            startIndex = cumuDecodeIndex[i-1] 
        endIndex = cumuDecodeIndex[i]  

        substring = ind[startIndex:endIndex]
        chromosome.append(decode(substring))
    return chromosome

def slotsAvaible(c):
    slotsUsed = []
    for i in c:
        d = decodeChromosome(i)
        slotsUsed.append([d[5],d[6],d[7]])
        slotsUsed.append([d[9],d[10],d[11]])
    allslots = []
    for d in range(NUM_DAYS):
        for t in range(NUM_TIMESLOTS):
            for r in range(NUM_ROOMS):
                slot = [d, t, r]
                if slot not in slotsUsed:
                    allslots.append(slot)
    return allslots
    
def createChromosome():
    chromosome = []
    for i in range(NUM_COURSES):
        for j in range(NUM_SECTIONS):
            freeSlots = slotsAvaible(chromosome)
            free = [slot for slot in freeSlots if slot[0] < 3]
            free = [slot for slot in free if slot[1] < 4]
            selectedSlot = random.choice(free)
            rtype = genNum(0,1)
            time1 = selectedSlot[1]
            room1 = selectedSlot[2]
            time2 = time1
            room2 = room1
            day1 = selectedSlot[0]
            day2 = selectedSlot[0]+2
            chromosome.append(encodeChromosome([i,0,j,genNum(20,120),i,day1,time1,room1,rtype,day2,time2,room2,rtype]))
            
    for i in range(NUM_LABS):
        for j in range(NUM_SECTIONS):
            freeSlots = slotsAvaible(chromosome)
            free = [slot for slot in freeSlots if slot[1] == 4]
            selectedSlot = random.choice(free)
            rtype = genNum(0,1)
            day = selectedSlot[0]
            time = selectedSlot[1]
            room = selectedSlot[2]
            chromosome.append(encodeChromosome([i,1,j,genNum(20,120),i,day,time,room,rtype,day,time+1,room,rtype]))
    
    return chromosome

def generatePopulation(size):
    population = []
    for _ in range(size):
        i = createChromosome()
        population.append(i)
    return population


In [3]:
def roomTimeCheck(List):
    value = 0
    for i in List:
        for j in List:
            if i[cumuDecodeIndex[4]:cumuDecodeIndex[8]] == j[cumuDecodeIndex[4]:cumuDecodeIndex[8]]:
                value += 1
            if i[cumuDecodeIndex[8]:cumuDecodeIndex[12]] == j[cumuDecodeIndex[8]:cumuDecodeIndex[12]]:
                value += 1
    #print("roomConfilct",value)        
    return value

def professorConfilct(chromosome):
    value = 0
    
    for p in range(NUM_PROFESSORS):
        professorSlots = []
        prof = encode(p,decodeIndex[4])
        for i in chromosome:
            if prof == i[cumuDecodeIndex[3]:cumuDecodeIndex[4]]:
                professorSlots.append(i)
        value += roomTimeCheck(professorSlots)
        
    #print("professorConfilct",value)
    return value

def sameSectionConfilct(chromosome):
    value = 0
    #for i in chromosome:
    #    if 
    return value

def lectureGapConfilct(chromosome):
    value = 0
    for i in chromosome:
        if i[cumuDecodeIndex[0]:cumuDecodeIndex[1]] == 0:
            if i[cumuDecodeIndex[4]:cumuDecodeIndex[5]] == encode(int(decode(i[cumuDecodeIndex[8]:cumuDecodeIndex[9]])) - 1) or i[cumuDecodeIndex[4]:cumuDecodeIndex[5]] == i[cumuDecodeIndex[8]:cumuDecodeIndex[9]]:
                value += 1
    #print("lectureConfilct",value)
    return value
# NUM_COURSES,1 ,NUM_SECTIONS, 120 ,NUM_PROFESSORS, NUM_DAYS, NUM_TIMESLOTS, NUM_ROOMS
#[4, 5, 8, 19, 23, 26, 29, 32, 33, 36, 39, 42, 43]
def labLectureConfilct(chromosome):
    value = 0
    for i in chromosome:
        if i[cumuDecodeIndex[0]:cumuDecodeIndex[1]] == 1:
            if i[cumuDecodeIndex[4]:cumuDecodeIndex[5]] != i[cumuDecodeIndex[8]:cumuDecodeIndex[9]]:
                value += 1
            elif i[cumuDecodeIndex[6]:cumuDecodeIndex[7]] != i[cumuDecodeIndex[10]:cumuDecodeIndex[11]]:
                value += 1
            elif i[cumuDecodeIndex[5]:cumuDecodeIndex[6]] != encode(int(decode(i[cumuDecodeIndex[9]:cumuDecodeIndex[10]])) - 1):
                value += 1
    #print("labConfilct",value)
    return value

def calculateFitness(chromosome):
    confilcts = 0
    confilcts += professorConfilct(chromosome) # A professor should not be assigned two different lectures at the same time.
    confilcts += sameSectionConfilct(chromosome) # The same section cannot be assigned to two different rooms at the same time.
    confilcts += lectureGapConfilct(chromosome)  # Each course would have two lectures per week not on the same or adjacent days.
    confilcts += labLectureConfilct(chromosome)  # Lab lectures should be conducted in two consecutive slots.
    return -confilcts

## Selection
### Select two chromosomes from a population for crossover.

In [4]:
def selectChromosomes(population):
    fitness_values = []
    for i in population:
        f = calculateFitness(i)  # Assuming calculateFitness function calculates fitness values
        fitness_values.append(f)
    
    m = max(fitness_values)
    if m == 0:
        return population[fitness_values.index(m)], []
    fit = fitness_values
    #print(sum(fit))
    fitness_values = [-1*(1/i) for i in fitness_values]
    #print(fit)
    #print(fitness_values)
    parent1 = random.choices(population, weights=fitness_values, k=1)[0]
    parent2 = random.choices(population, weights=fitness_values, k=1)[0]
    return parent1, parent2


## CrossOver
### It takes two parent chromosomes as input and randomly selects a crossover point.

In [5]:
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1))
    child1 = parent1[0:crossover_point] + parent2[crossover_point:]
    child2 = parent2[0:crossover_point] + parent1[crossover_point:]

    #print("Performed crossover between two chromosomes",child1,child2)
    return child1, child2

## Mutation
### performs a mutation on a chromosome

In [6]:
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome)-1)
    while True:
        chromo = (random.randint(0,5),random.randint(0,5))
        if chromo not in chromosome:
            break;

    chromosome[mutation_point] = chromo
    print("Performed mutation on a chromosome")
    return chromosome

## Get Best Chromosomes
### This function takes in a population of chromosomes and returns the best chromosome from the population.

In [7]:
def get_best(population):
    fitness_values = []
    for chromosome in population:
        fitness_values.append(calculateFitness(chromosome))

    max_value = max(fitness_values)
    max_index = fitness_values.index(max_value)
    return population[max_index]

In [8]:
def printTimeTable(chromosome):
    c = []
    for i in chromosome:
        ch = decodeChromosome(i)
        c.append(ch)
    
    print("Timetable Format:")
    for d in range(len(DAYS)):
        print(DAYS[d])
        print("Rooms\t", end="")
        max_width = max(len(timeslot) for timeslot in TIMESLOTS) + 2
        for timeslot in TIMESLOTS:
            print(f"{timeslot: <{max_width}}", end="")
        print()
        
        for r in range(len(ROOMS_NAMES)):
            print(ROOMS_NAMES[r], end="\t")
            for t in range(NUM_TIMESLOTS):
                found_course = False
                for n in c:
                    if (n[5] == d and n[6] == t and n[7] == r) or (n[9] == d and n[10] == t and n[11] == r):
                        course_str = f"{COURSE_NAMES[n[0]]}({SECTIONS[n[2]]})"
                        print(f"{course_str: <{max_width}}", end="")
                        found_course = True
                        break
                if not found_course:
                    print("    ".ljust(max_width), end="")
            print()
        print()

In [9]:
population_size = 20
mutation_probability = 0
generations = 100
# generate a random population
population = generatePopulation(population_size)
#print(calculateFitness(population[0]))
fitness = 10
for _ in range(generations):
    # select two chromosomes for crossover
    parent1, parent2 = selectChromosomes(population)
    
    if len(parent2) == 0:
        break
    # perform crossover to generate two new chromosomes
    child1, child2 = crossover(parent1, parent2)
    
    # perform mutation on the two new chromosomes
    if random.uniform(0, 1) < mutation_probability:
        child1 = mutate(child1)
    if random.uniform(0, 1) < mutation_probability:
        child2 = mutate(child2)
    
    # replace the old population with the new population
    fitness_values = []
    for chromosome in population:
        fitness_values.append(calculateFitness(chromosome))
    index = np.argmin(fitness_values)
    population[index] = child1
    index = np.argmin(fitness_values)
    population[index] = child2
    best = get_best(population)
    fitness = calculateFitness(best)
    

best = get_best(population)
#print(best)
printTimeTable(best)

Timetable Format:
Monday
Rooms	8:30-9:50    10:00-11:20  11:30-12:50  1:00-2:20    2:30-3:50    4:00-5:15    
C-301	             CALCULAS(E)                            OS(E)        OS(E)        
C-302	AI(E)                     WEB(C)       OS(A)        IICT(A)      IICT(A)      
C-303	TBW(B)       Algo(B)                   PF(B)        ML(A)        ML(A)        
C-304	DB(D)        IICT(B)      TBW(E)       TBW(D)                                 
C-401	PROB(C)      ML(E)                                  OS(D)        OS(D)        
C-402	             PAKSTD(A)                 PF(A)        ML(C)        ML(C)        
C-403	                          OS(C)        ML(B)        OS(B)        OS(B)        
C-404	IICT(D)      ML(A)        AI(D)        ML(C)                                  

Tuesday
Rooms	8:30-9:50    10:00-11:20  11:30-12:50  1:00-2:20    2:30-3:50    4:00-5:15    
C-301	PF(E)                                                                         
C-302	WEB(E)       CALCULAS(A) 